#### Imports

In [ ]:
import pandas # Pour la manipulation des données
from sklearn import tree # Pour les arbres de décision
from sklearn.tree import DecisionTreeClassifier # Pour les arbres de décision
import matplotlib.pyplot as plt # Pour les graphiques

# On load le dataset
data = pandas.read_csv("listings.csv")

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2719440,Nice flat close to Montparnasse,13915159,Thibaut,NaN,Palais-Bourbon,48.849970,2.319740,Entire home/apt,NaN,2,79,2020-03-13,0.57,1,0,0,7510702406383
1,2721397,"Marais - Charming loft, river view",13925876,Cecilie,NaN,Hôtel-de-Ville,48.852030,2.361760,Entire home/apt,NaN,5,88,2025-08-29,0.66,1,44,13,7510400455677
2,2726092,Superbe 2 pièces idéalement placé,13612373,Clemence,NaN,Batignolles-Monceau,48.880040,2.306860,Entire home/apt,NaN,365,0,NaN,NaN,1,0,0,NaN
3,2726327,"Cozy place, Sacré Coeur Montmartre in the SoPi",7483725,Hélène,NaN,Opéra,48.882710,2.343550,Entire home/apt,NaN,3,99,2025-07-03,0.72,1,297,8,7510901014972
4,2727929,Appartement d'architecte - Montmartre,13960346,Paul,NaN,Buttes-Montmartre,48.894917,2.336131,Entire home/apt,NaN,2,7,2025-07-20,0.25,1,83,1,7511808333770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81848,1481754230424845668,Apt a/c 1BR/2P - Saint-Lazare/Les Champs Elysée,524851930,Joffrey Sally,NaN,Élysée,48.875271,2.325716,Entire home/apt,NaN,3,2,2025-09-14,2.00,184,346,2,7510815621419
81849,1481771421372168466,Superbe duplex avec terrasse face Tour Eiffel,563248085,Eliav,NaN,Vaugirard,48.853720,2.296581,Entire home/apt,NaN,30,0,NaN,NaN,76,250,0,"Available with a mobility lease only (""bail mo..."
81850,1481789891762355898,Cozy studio - 2P - Place des Vosges/Marais,691591816,Michael David,NaN,Hôtel-de-Ville,48.854545,2.363546,Entire home/apt,NaN,2,1,2025-09-07,1.00,3,322,1,7510406440280
81851,1481814132906153972,Appt entier 35m2 proche Montmartre,35944108,Lucie,NaN,Opéra,48.878290,2.340380,Private room,NaN,1,0,NaN,NaN,2,23,0,NaN


#### Collecte de données
afficher nombre de colonnes et de lignes


## Pré-traitement des données

#### Sélection des colonnes les plus pertinentes à évaluer

#### Supprimer les doublons

#### Conversion des types de colonnes
- Conversion de Texte en Chiffres
- Extraction de Nombres
- Uniformisation des Données

## Analyse descriptive des données

#### Relation entre la note globale et les prix des locations à Paris

#### Prix moyen par type de propriété à Paris

#### Prix en fonction de Location disponible à Paris

#### Répartition des locations disponible et non disponible pour Paris

## Gestion des valeurs abberantes

## Séparation des données en ensemble d'entraînement et de test

#### Régression linéaire simple

#### Régression linéaire multiple

#### Decision tree